In [1]:
import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)
time.sleep(1) # Wait for the camera to initialize
_, frame = cap.read()
x, y, w, h = cv2.selectROI(frame)
track_window = (x, y, w, h)

roi = frame[y:y + h, x:x + w]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        pts = cv2.boxPoints(ret)
        #print(pts)
        pts = np.int0(pts)
        print(pts)
        cv2.polylines(frame, [pts], True, (0, 255, 0), 2)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

[[253 262]
 [448 266]
 [445 400]
 [250 396]]
[[218 273]
 [461 262]
 [469 435]
 [226 446]]
[[182 259]
 [481 254]
 [484 467]
 [185 472]]
[[158 240]
 [517 251]
 [510 495]
 [151 484]]
[[125 240]
 [543 253]
 [535 495]
 [117 482]]
[[ 95 245]
 [563 251]
 [560 492]
 [ 92 486]]
[[ 58 243]
 [574 253]
 [569 496]
 [ 53 486]]
[[ 43 248]
 [585 253]
 [582 497]
 [ 40 492]]
[[ 42 249]
 [588 257]
 [585 496]
 [ 39 488]]
[[ 40 248]
 [595 261]
 [590 497]
 [ 35 484]]
[[ 35 254]
 [595 255]
 [594 495]
 [ 34 494]]
[[ 35 252]
 [598 260]
 [595 497]
 [ 32 489]]
[[ 33 256]
 [598 258]
 [597 491]
 [ 32 489]]
[[ 31 250]
 [599 260]
 [594 497]
 [ 26 487]]
[[ 35 252]
 [597 257]
 [595 495]
 [ 33 490]]
[[ 34 255]
 [594 254]
 [595 492]
 [ 35 493]]
[[ 34 254]
 [596 255]
 [595 491]
 [ 33 490]]
[[ 36 247]
 [596 257]
 [591 498]
 [ 31 488]]
[[ 32 248]
 [594 256]
 [591 497]
 [ 29 489]]
[[ 31 251]
 [597 254]
 [596 490]
 [ 30 487]]
[[ 40 247]
 [601 257]
 [596 498]
 [ 35 488]]
[[ 36 254]
 [595 253]
 [596 489]
 [ 37 490]]
[[ 30 253]

In [2]:
import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)  
# 0 for default camera
time.sleep(1)
_, frame = cap.read()
x, y, w, h = cv2.selectROI(frame)
track_window = (x, y, w, h)

roi = frame[y:y + h, x:x + w] #put frame Size 
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        center = (pts[0] + pts[2]) // 2
        angle = ret[2]
        cv2.polylines(frame, [pts], True, (0, 255, 0), 2)
        cv2.circle(frame, tuple(center), 5, (0, 0, 255), -1)
        cv2.putText(frame, f"Angle: {angle:.2f}", tuple(center), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        print(pts)
        print(f"Angle: {angle:.2f}")
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == ord('q'):
            break

# Release the resources
cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)  
# 0 for default camera
time.sleep(1)
_, frame = cap.read()
x, y, w, h = cv2.selectROI(frame)
track_window = (x, y, w, h)

# initialize previous centroid
prev_centroid = (x + w/2, y + h/2)

roi = frame[y:y + h, x:x + w] #put frame Size 

hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)
while cap.isOpened():
    ret, frame = cap.read()
    if ret== True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        center = (pts[0] + pts[2]) // 2
        angle = ret[2]
        cv2.polylines(frame, [pts], True, (0, 255, 0), 2)
        cv2.circle(frame, tuple(center), 5, (0, 0, 255), -1)
        cv2.putText(frame, f"Angle: {angle:.2f}", tuple(center), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        #print(pts)
        print(f"Angle is: {angle:.2f}")
        print(rf"points is {pts}")
        cv2.polylines(frame, [pts], True, (0, 255, 0), 2)
        centroid = (int(ret[0][0]), int(ret[0][1]))
        
        # calculate velocity based on previous and current centroids
        velocity = (centroid[0] - prev_centroid[0], centroid[1] - prev_centroid[1])
        print("Velocity is:", velocity)
        
        prev_centroid = centroid
        
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == ord('q'):
            break

 # Release the resources
cap.release()
cv2.destroyAllWindows()    


Angle is: 68.60
points is [[164 169]
 [272 127]
 [312 229]
 [204 271]]
Velocity is: (-21.0, -1.0)
Angle is: 102.99
points is [[182 118]
 [310 147]
 [283 263]
 [155 234]]
Velocity is: (-5, -8)
Angle is: 101.33
points is [[187 114]
 [344 145]
 [319 270]
 [162 239]]
Velocity is: (20, 1)
Angle is: 108.00
points is [[189 103]
 [353 156]
 [310 289]
 [146 236]]
Velocity is: (-3, 4)
Angle is: 109.52
points is [[191  92]
 [361 153]
 [311 294]
 [141 233]]
Velocity is: (1, -3)
Angle is: 124.34
points is [[218  64]
 [372 169]
 [281 303]
 [127 198]]
Velocity is: (-1, -9)
Angle is: 115.03
points is [[200  85]
 [363 161]
 [299 297]
 [136 221]]
Velocity is: (0, 7)
Angle is: 119.97
points is [[210  75]
 [368 166]
 [290 301]
 [132 210]]
Velocity is: (0, -3)
Angle is: 109.92
points is [[194  94]
 [361 154]
 [309 298]
 [142 238]]
Velocity is: (2, 8)
Angle is: 122.10
points is [[217  73]
 [372 170]
 [285 309]
 [130 212]]
Velocity is: (-1, -5)
Angle is: 132.65
points is [[235  63]
 [365 183]
 [257 300]
 [12

Velocity is: (-1, -5)
Angle is: 174.25
points is [[317  49]
 [344 309]
 [200 324]
 [173  64]]
Velocity is: (1, 4)
Angle is: 171.11
points is [[313  46]
 [354 305]
 [206 328]
 [165  69]]
Velocity is: (1, 0)
Angle is: 173.19
points is [[313  47]
 [344 310]
 [203 327]
 [172  64]]
Velocity is: (-2, 0)
Angle is: 173.13
points is [[314  43]
 [346 311]
 [204 328]
 [172  60]]
Velocity is: (1, -1)
Angle is: 174.02
points is [[316  43]
 [345 310]
 [203 325]
 [174  58]]
Velocity is: (1, -2)
Angle is: 174.00
points is [[317  45]
 [346 315]
 [204 330]
 [175  60]]
Velocity is: (1, 4)
Angle is: 174.73
points is [[321  49]
 [345 310]
 [200 323]
 [176  62]]
Velocity is: (0, -2)
Angle is: 172.72
points is [[314  47]
 [347 311]
 [203 330]
 [170  66]]
Velocity is: (-2, 3)
Angle is: 173.23
points is [[315  45]
 [346 304]
 [202 321]
 [171  62]]
Velocity is: (0, -6)
Angle is: 174.40
points is [[321  41]
 [347 306]
 [199 321]
 [173  56]]
Velocity is: (1, -2)
Angle is: 172.97
points is [[315  46]
 [348 307]
 [

Velocity is: (13, 0)
Angle is: 168.29
points is [[311  30]
 [362 279]
 [225 307]
 [174  58]]
Velocity is: (14, -2)
Angle is: 177.67
points is [[360  51]
 [369 290]
 [217 296]
 [208  57]]
Velocity is: (21, 5)
Angle is: 10.60
points is [[209 288]
 [254  47]
 [417  78]
 [372 319]]
Velocity is: (24, 9)
Angle is: 15.19
points is [[217 282]
 [281  46]
 [447  91]
 [383 327]]
Velocity is: (19, 4)
Angle is: 20.00
points is [[217 278]
 [300  49]
 [462 108]
 [379 337]]
Velocity is: (8, 6)
Angle is: 19.64
points is [[216 281]
 [299  49]
 [459 106]
 [376 338]]
Velocity is: (-2, 1)
Angle is: 18.24
points is [[215 283]
 [292  48]
 [451 101]
 [374 336]]
Velocity is: (-5, -2)
Angle is: 20.36
points is [[201 268]
 [283  47]
 [444 107]
 [362 328]]
Velocity is: (-10, -4)
Angle is: 11.09
points is [[200 278]
 [245  52]
 [410  85]
 [365 311]]
Velocity is: (-18, -6)
Angle is: 178.79
points is [[366  62]
 [371 298]
 [205 301]
 [200  65]]
Velocity is: (-19, 0)
Angle is: 167.39
points is [[324  42]
 [377 279]
 

Angle is: 178.87
points is [[379  70]
 [384 305]
 [219 308]
 [214  73]]
Velocity is: (0, -1)
Angle is: 177.50
points is [[373  65]
 [384 299]
 [219 306]
 [208  72]]
Velocity is: (-3, -3)
Angle is: 178.13
points is [[376  69]
 [384 306]
 [216 311]
 [208  74]]
Velocity is: (0, 4)
Angle is: 173.55
points is [[368  64]
 [394 292]
 [226 311]
 [200  83]]
Velocity is: (1, -2)
Angle is: 176.72
points is [[373  68]
 [386 297]
 [218 306]
 [205  77]]
Velocity is: (-1, -1)
Angle is: 0.81
points is [[213 304]
 [216  68]
 [382  71]
 [379 307]]
Velocity is: (2, 1)
Angle is: 0.60
points is [[211 304]
 [214  65]
 [379  66]
 [376 305]]
Velocity is: (-3, -3)
Angle is: 177.79
points is [[371  60]
 [380 296]
 [217 303]
 [208  67]]
Velocity is: (-1, -3)
Angle is: 175.59
points is [[363  51]
 [382 297]
 [222 309]
 [203  63]]
Velocity is: (-1, -2)
Angle is: 178.31
points is [[370  50]
 [377 299]
 [218 304]
 [211  55]]
Velocity is: (1, -3)
Angle is: 176.67
points is [[368  44]
 [383 304]
 [221 313]
 [206  53]]

Velocity is: (2, 2)
Angle is: 170.23
points is [[323  33]
 [368 291]
 [216 317]
 [171  59]]
Velocity is: (0, -1)
Angle is: 171.11
points is [[323  30]
 [365 296]
 [216 319]
 [174  53]]
Velocity is: (0, 0)
Angle is: 170.69
points is [[318  37]
 [359 289]
 [214 313]
 [173  61]]
Velocity is: (-4, 0)
Angle is: 170.73
points is [[314  38]
 [355 290]
 [215 313]
 [174  61]]
Velocity is: (-1, 1)
Angle is: 169.20
points is [[313  34]
 [362 292]
 [221 319]
 [172  61]]
Velocity is: (2, 1)
Angle is: 171.46
points is [[319  30]
 [359 296]
 [212 318]
 [172  52]]
Velocity is: (-1, -3)
Angle is: 170.54
points is [[314  25]
 [359 292]
 [214 316]
 [169  49]]
Velocity is: (-2, -3)
Angle is: 169.60
points is [[313  31]
 [361 293]
 [218 320]
 [170  58]]
Velocity is: (2, 5)
Angle is: 168.74
points is [[305  34]
 [356 290]
 [216 317]
 [165  61]]
Velocity is: (-5, 0)
Angle is: 169.58
points is [[308  26]
 [357 292]
 [217 318]
 [168  52]]
Velocity is: (2, -4)
Angle is: 168.36
points is [[297  25]
 [353 296]
 [

In [6]:
import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)  
# 0 for default camera
time.sleep(1)
_, frame = cap.read()
x, y, w, h = cv2.selectROI(frame)
track_window = (x, y, w, h)

# Initialize position variables
prev_center = None
prev_frame_time = time.time()

roi = frame[y:y + h, x:x + w] #put frame Size 
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while cap.isOpened():
    ret, frame = cap.read()
    if ret== True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        center = (pts[0] + pts[2]) // 2
        angle = ret[2]
        cv2.polylines(frame, [pts], True, (0, 255, 0), 2)
        cv2.circle(frame, tuple(center), 5, (0, 0, 255), -1)
        cv2.putText(frame, f"Angle: {angle:.2f}", tuple(center), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        #print(pts)
        print(f"Angle is: {angle:.2f}")
        print(rf"points is {pts}")
        center = tuple(np.mean(pts, axis=0, dtype=np.int))
        cv2.polylines(frame, [pts], True, (0, 255, 0), 2)
        cv2.circle(frame, center, 3, (0, 0, 255), -1)
        cv2.imshow('frame', frame)
        
        # Calculate velocity
        if prev_center is not None:
            curr_frame_time = time.time()
            time_diff = curr_frame_time - prev_frame_time
            prev_frame_time = curr_frame_time
            position_diff = np.subtract(center, prev_center)
            velocity = position_diff / time_diff
            print(f"Velocity: {velocity}")
        
        prev_center = center
        
        if cv2.waitKey(1) == ord('q'):
            break

# Release the resources
cap.release()
cv2.destroyAllWindows()


Angle is: 134.13
points is [[251 150]
 [291 189]
 [256 226]
 [216 187]]
Angle is: 70.45
points is [[198 158]
 [264 135]
 [284 192]
 [218 215]]
Velocity: [-230.79019094 -250.02270685]
Angle is: 78.96
points is [[173 151]
 [268 132]
 [284 213]
 [189 232]]
Velocity: [-812.31678684  437.40134676]
Angle is: 113.39
points is [[196 119]
 [305 166]
 [267 253]
 [158 206]]
Velocity: [249.96348755 333.28465007]
Angle is: 100.95
points is [[187 124]
 [320 149]
 [300 253]
 [167 228]]
Velocity: [857.19038779 142.86506463]
Angle is: 103.50
points is [[192 111]
 [340 146]
 [311 267]
 [163 232]]
Velocity: [500.12567817  62.51570977]
Angle is: 102.80
points is [[194 102]
 [341 135]
 [310 273]
 [163 240]]
Velocity: [ 38.44634493 -76.89268986]
Angle is: 159.42
points is [[287  63]
 [350 229]
 [215 280]
 [152 114]]
Velocity: [ -32.26586251 -516.25380023]
Angle is: 167.45
points is [[294  58]
 [337 248]
 [200 279]
 [157  89]]


C:\Users\moels\AppData\Local\Temp/ipykernel_15324/1730805011.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  center = tuple(np.mean(pts, axis=0, dtype=np.int))


Velocity: [-117.63908678  -88.22931508]
Angle is: 166.86
points is [[293  55]
 [338 247]
 [202 279]
 [157  87]]
Velocity: [  0.         -32.25618506]
Angle is: 167.45
points is [[293  56]
 [338 257]
 [204 287]
 [159  86]]
Velocity: [ 29.2653084 117.0612336]
Angle is: 171.70
points is [[299  58]
 [329 263]
 [193 282]
 [163  77]]
Velocity: [-68.97618734 -34.48809367]
Angle is: 171.46
points is [[302  55]
 [333 265]
 [195 286]
 [164  76]]
Velocity: [60.63104333  0.        ]
Angle is: 168.19
points is [[293  55]
 [335 255]
 [201 283]
 [159  83]]
Velocity: [-22.21983005 -22.21983005]
Angle is: 168.72
points is [[295  53]
 [336 260]
 [201 287]
 [160  80]]
Velocity: [30.29975366 30.29975366]
Angle is: 170.95
points is [[301  57]
 [334 263]
 [194 285]
 [161  79]]
Velocity: [-30.31508344  30.31508344]
Angle is: 170.22
points is [[300  58]
 [335 262]
 [195 286]
 [160  82]]
Velocity: [ 0.         32.26313239]
Angle is: 168.68
points is [[292  53]
 [333 261]
 [198 288]
 [157  80]]
Velocity: [-64.5

Velocity: [  0.         -31.24993667]
Angle is: 157.24
points is [[264  46]
 [345 239]
 [216 293]
 [135 100]]
Velocity: [-181.80902911   30.30150485]
Angle is: 157.09
points is [[258  43]
 [344 246]
 [219 298]
 [133  95]]
Velocity: [-66.66884959  33.3344248 ]
Angle is: 154.54
points is [[257  44]
 [348 234]
 [221 295]
 [130 105]]
Velocity: [ 28.57174776 -28.57174776]
Angle is: 152.52
points is [[249  45]
 [347 233]
 [223 297]
 [125 109]]
Velocity: [-103.44811937   68.96541292]
Angle is: 157.86
points is [[260  57]
 [339 250]
 [212 301]
 [133 108]]
Velocity: [  0.         250.01066969]
Angle is: 147.98
points is [[226  47]
 [340 229]
 [227 300]
 [113 118]]
Velocity: [-303.01723764 -181.81034259]
Angle is: 150.97
points is [[232  48]
 [337 237]
 [220 302]
 [115 113]]
Velocity: [ 0.         62.48218329]
Angle is: 157.14
points is [[256  70]
 [331 249]
 [215 298]
 [140 119]]
Velocity: [290.37042507 290.37042507]
Angle is: 155.11
points is [[249  72]
 [331 249]
 [218 301]
 [136 124]]
Veloci

Velocity: [-64.50244904  96.75367356]
Angle is: 159.53
points is [[262  44]
 [339 250]
 [210 298]
 [133  92]]
Velocity: [ -62.50918792 -125.01837583]
Angle is: 157.82
points is [[258  39]
 [342 245]
 [213 298]
 [129  92]]
Velocity: [-32.25891401 -96.77674204]
Angle is: 157.12
points is [[254  44]
 [341 250]
 [217 303]
 [130  97]]
Velocity: [  0.         104.17473461]
Angle is: 159.43
points is [[264  49]
 [340 251]
 [210 300]
 [134  98]]
Velocity: [62.49521709 31.24760855]
Angle is: 159.45
points is [[263  47]
 [340 250]
 [210 299]
 [133  96]]
Velocity: [-31.24504801 -31.24504801]
Angle is: 159.93
points is [[261  43]
 [337 250]
 [206 298]
 [130  91]]
Velocity: [-90.91568041 -90.91568041]
Angle is: 159.54
points is [[261  44]
 [340 254]
 [211 302]
 [132  92]]
Velocity: [90.90451455 90.90451455]
Angle is: 160.67
points is [[264  43]
 [339 257]
 [210 302]
 [135  88]]
Velocity: [ 33.14581045 -33.14581045]
Angle is: 156.79
points is [[252  43]
 [341 250]
 [215 304]
 [126  97]]
Velocity: [-

...

Angle is: 167.14
points is [[301  11]
 [362 276]
 [206 311]
 [145  46]]
Velocity: [63.28980526 12.65796105]
Angle is: 166.93
points is [[302   9]
 [364 276]
 [206 313]
 [144  46]]
Velocity: [90.90976873  0.        ]
Angle is: 165.23
points is [[288   1]
 [361 279]
 [212 318]
 [139  40]]
Velocity: [-190.49215991  -95.24607996]
Angle is: 167.69
points is [[310   8]
 [371 285]
 [210 320]
 [149  43]]
Velocity: [312.48772565 156.24386283]
Angle is: 167.30
points is [[322  14]
 [383 285]
 [215 323]
 [154  52]]
Velocity: [249.99949336 124.99974668]
Angle is: 167.46
points is [[358  30]
 [415 286]
 [227 327]
 [170  71]]
Velocity: [727.24662433 303.0194268 ]
Angle is: 2.59
points is [[198 303]
 [209  55]
 [423  64]
 [412 312]]
Velocity: [580.66938424 161.29705118]
Angle is: 9.31
points is [[180 288]
 [220  45]
 [439  81]
 [399 324]]
Velocity: [-30.27394709  30.27394709]
Angle is: 5.03
points is [[186 291]
 [208  44]
 [416  63]
 [394 310]]
Velocity: [-250.24187101 -218.96163713]
Angle is: 175.17

Velocity: [ 31.24853975 -62.49707951]
Angle is: 161.61
points is [[313  12]
 [396 262]
 [238 315]
 [155  65]]
Velocity: [  0.         124.94668404]
Angle is: 158.11
points is [[296   9]
 [393 250]
 [245 309]
 [148  68]]
Velocity: [-146.97568804 -117.58055044]
Angle is: 161.51
points is [[303  12]
 [385 257]
 [236 306]
 [154  61]]
Velocity: [-31.23853218   0.        ]
Angle is: 163.84
points is [[310  15]
 [381 259]
 [231 302]
 [160  58]]
Velocity: [ 32.30413284 -32.30413284]
Angle is: 162.55
points is [[311   8]
 [390 260]
 [236 309]
 [157  57]]
Velocity: [62.49428593  0.        ]
Angle is: 165.16
points is [[315   2]
 [385 266]
 [230 307]
 [160  43]]
Velocity: [ -32.26114713 -129.04458853]
Angle is: 162.05
points is [[314   2]
 [399 265]
 [242 316]
 [157  53]]
Velocity: [181.83530228 151.52941857]
Angle is: 159.62
points is [[308  11]
 [399 256]
 [244 313]
 [153  68]]
Velocity: [-66.66249195  99.99373793]
Angle is: 160.37
points is [[299   2]
 [390 256]
 [240 309]
 [149  55]]
Velocity

Angle is: 171.02
points is [[336  26]
 [373 262]
 [223 285]
 [186  49]]
Velocity: [31.24900538  0.        ]
Angle is: 169.34
points is [[333  26]
 [377 258]
 [227 287]
 [183  55]]
Velocity: [31.24900538 31.24900538]
Angle is: 169.61
points is [[336  30]
 [377 254]
 [226 282]
 [185  58]]
Velocity: [24.38804061  0.        ]
Angle is: 172.23
points is [[343  34]
 [374 261]
 [220 282]
 [189  55]]
Velocity: [ 0.         57.32938772]
Angle is: 167.85
points is [[331  31]
 [380 258]
 [230 291]
 [181  64]]
Velocity: [-16.94907967  50.847239  ]
Angle is: 171.75
points is [[342  35]
 [375 265]
 [219 288]
 [186  58]]
Velocity: [0. 0.]
Angle is: 175.53
points is [[351  39]
 [368 264]
 [214 276]
 [197  51]]
Velocity: [  79.90748626 -159.81497252]
Angle is: 174.11
points is [[346  42]
 [369 268]
 [219 283]
 [196  57]]
Velocity: [  0.        278.2401953]
Angle is: 177.03
points is [[351  51]
 [362 270]
 [210 278]
 [199  59]]
Velocity: [-42.54980015  42.54980015]
Angle is: 179.76
points is [[357  55]


Velocity: [-30.46525513 -91.39576539]
Angle is: 179.29
points is [[332  67]
 [335 272]
 [195 273]
 [192  68]]
Velocity: [34.48582517  0.        ]
Angle is: 177.75
points is [[330  64]
 [339 273]
 [197 278]
 [188  69]]
Velocity: [ 0.         31.25273088]
Angle is: 0.07
points is [[192 271]
 [193  72]
 [336  72]
 [335 271]]
Velocity: [30.3021616  0.       ]
Angle is: 177.67
points is [[333  66]
 [341 269]
 [194 275]
 [186  72]]
Velocity: [-32.23337919 -32.23337919]
Angle is: 179.32
points is [[333  67]
 [336 271]
 [193 272]
 [190  68]]
Velocity: [  0.         -31.23783421]
Angle is: 179.85
points is [[337  69]
 [338 272]
 [193 273]
 [192  70]]
Velocity: [62.17329884 62.17329884]
Angle is: 178.68
points is [[332  65]
 [337 271]
 [194 274]
 [189  68]]
Velocity: [-60.69640971 -60.69640971]
Angle is: 178.35
points is [[331  62]
 [337 275]
 [196 279]
 [190  66]]
Velocity: [ 0.         32.26213973]
Angle is: 0.18
points is [[194 274]
 [195  67]
 [334  68]
 [333 275]]
Velocity: [21.21571285 21.

Velocity: [  0.         -90.91042555]
Angle is: 165.49
points is [[288  56]
 [337 247]
 [198 283]
 [149  92]]
Velocity: [  0.         -62.50825633]
Angle is: 166.19
points is [[289  56]
 [336 249]
 [196 284]
 [149  91]]
Velocity: [-21.73743865  21.73743865]
Angle is: 168.11
points is [[293  61]
 [334 255]
 [194 284]
 [153  90]]
Velocity: [30.30610269 60.61220538]
Angle is: 166.54
points is [[294  60]
 [340 252]
 [196 287]
 [150  95]]
Velocity: [64.51683561 32.25841781]
Angle is: 167.50
points is [[295  59]
 [337 250]
 [194 281]
 [152  90]]
Velocity: [-30.30172376 -90.90517129]
Angle is: 161.90
points is [[281  52]
 [343 240]
 [205 285]
 [143  97]]
Velocity: [-32.255937 -64.511874]
Angle is: 166.68
points is [[288  57]
 [335 252]
 [199 284]
 [152  89]]
Velocity: [ 0.         62.42731481]
Angle is: 166.21
points is [[291  60]
 [338 251]
 [199 285]
 [152  94]]
Velocity: [62.50918792 62.50918792]
Angle is: 166.44
points is [[292  56]
 [339 250]
 [199 284]
 [152  90]]
Velocity: [  0.       

In [2]:
import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)  
time.sleep(1)
_, frame = cap.read()
x, y, w, h = cv2.selectROI(frame)
track_window = (x, y, w, h)

# Initial position of the object
x0, y0, z0 = x + w/2, y + h/2, 0

roi = frame[y:y + h, x:x + w] #put frame Size 
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        #print(pts)
        cv2.polylines(frame, [pts], True, (0, 255, 0), 2)
        
        # Compute the position of the object
        x, y, w, h = cv2.boundingRect(pts)
        z = 100  # Assuming the focal length of the camera is 100 pixels
        
        # Compute the velocity towards the z-axis
        v_z = (z - z0) / 1  # Assuming the time difference is 1 second
        z0 = z
        print(v_z)
        
        # Compute the angle of the object in radians
        angle = np.arctan2(y + h/2 - y0, x + w/2 - x0)
        print(angle)
        
        # Update the position of the object
        x0, y0 = x + w/2, y + h/2
        
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == ord('q'):
            break

 # Release the resources
cap.release()
cv2.destroyAllWindows()


100.0
-2.9500909351201745
0.0
-2.0516836069902324
0.0
-1.6506263125071339
0.0
-1.6295521495106193
0.0
-1.6814535479687922
0.0
1.5083775167989393
0.0
1.446441332248135
0.0
-1.8157749899217608
0.0
1.4056476493802699
0.0
-2.2455372690184494
0.0
1.5707963267948966
0.0
1.5707963267948966
0.0
-2.356194490192345
0.0
-1.2490457723982544
0.0
2.819842099193151
0.0
0.0
0.0
0.0
0.0
2.356194490192345
0.0
0.4636476090008061
0.0
-0.6747409422235526
0.0
-1.695151321341658
0.0
0.9272952180016122
0.0
-1.1071487177940904
0.0
-0.348771003583907
0.0
0.0
0.0
2.356194490192345
0.0
-0.35877067027057225
0.0
0.7853981633974483
0.0
-0.9272952180016122
0.0
1.3521273809209546
0.0
-1.4056476493802699
0.0
-1.1071487177940904
0.0
1.5707963267948966
0.0
0.0
0.0
2.1112158270654806
0.0
0.0
0.0
-1.3258176636680326
0.0
1.0516502125483738
0.0
-2.279422598922567
0.0
1.5707963267948966
0.0
-1.5707963267948966
0.0
-2.8632929945846817
0.0
2.256525837701183
0.0
-2.0344439357957027
0.0
-2.356194490192345
0.0
0.5404195002705842
0

0.0
2.356194490192345
0.0
0.9272952180016122
0.0
-2.158798930342464
0.0
1.5707963267948966
0.0
-0.982793723247329
0.0
0.982793723247329
0.0
1.5707963267948966
0.0
1.5707963267948966
0.0
3.141592653589793
0.0
-0.24497866312686414
0.0
0.7266423406817256
0.0
0.24497866312686414
0.0
0.2662520491509254
0.0
1.4181469983996315
0.0
0.16514867741462683
0.0
0.960070362405688
0.0
0.7853981633974483
0.0
1.9513027039072615
0.0
-0.19739555984988075
0.0
-0.8519663271732721
0.0
3.141592653589793
0.0
0.6747409422235526
0.0
3.141592653589793
0.0
2.1112158270654806
0.0
-0.4636476090008061
0.0
1.5707963267948966
0.0
1.5707963267948966
0.0
0.0
0.0
2.896613990462929
0.0
0.0
0.0
1.7126933813990606
0.0
0.0
0.0
2.677945044588987
0.0
0.0
0.0
1.1071487177940904
0.0
1.8157749899217608
0.0
-2.9441970937399127
0.0
-1.0303768265243125
0.0
1.0303768265243125
0.0
-2.356194490192345
0.0
-1.2490457723982544
0.0
0.0
0.0
-1.5707963267948966
0.0
2.819842099193151
0.0
2.677945044588987
0.0
-0.5404195002705842
0.0
2.86329299

In [8]:
import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)  
time.sleep(1)
_, frame = cap.read()
x, y, w, h = cv2.selectROI(frame)
track_window = (x, y, w, h)

# Initial position of the object
x0, y0, z0 = x + w/2, y + h/2, 0

roi = frame[y:y + h, x:x + w] #put frame Size 
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        cv2.polylines(frame, [pts], True, (0, 255, 0), 2)
        
        # Compute the position of the object
        x, y, w, h = cv2.boundingRect(pts)
        z = 100  # Assuming the focal length of the camera is 100 pixels
        
        # Compute the velocity towards the z-axis
        v_z = (z - z0) / 1  # Assuming the time difference is 1 second
        z0 = z
        
        # Compute the angle of the object in radians
        angle = np.arctan2(y + h/2 - y0, x + w/2 - x0)
        
        # Update the position of the object
        x0, y0 = x + w/2, y + h/2
        
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == ord('q'):
            break

 # Release the resources
cap.release()
cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)  
time.sleep(1)
_, frame = cap.read()
x, y, w, h = cv2.selectROI(frame)
track_window = (x, y, w, h)

roi = frame[y:y + h, x:x + w]

hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        center = np.mean(pts, axis=0)
        cv2.polylines(frame, [pts], True, (0, 255, 0), 2)
        cv2.circle(frame, (int(center[0]), int(center[1])), 5, (0, 0, 255), -1)
        angle = np.arctan2(pts[1][1]-pts[0][1], pts[1][0]-pts[0][0])
        angle_deg = np.degrees(angle)
        print("Angle (in radians): ", angle)
        print("Angle (in degrees): ", angle_deg)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Angle (in radians):  -1.544206850142363
Angle (in degrees):  -88.47653520834818
Angle (in radians):  1.5481757514395829
Angle (in degrees):  88.70393650198288
Angle (in radians):  1.4911333417491095
Angle (in degrees):  85.43564717346261
Angle (in radians):  1.50066360449418
Angle (in degrees):  85.98169100640591
Angle (in radians):  1.4824195406500478
Angle (in degrees):  84.93638314696992
Angle (in radians):  1.4855334573543673
Angle (in degrees):  85.1147974318827
Angle (in radians):  1.4838789370912815
Angle (in degrees):  85.02000040368902
Angle (in radians):  1.4674315925719472
Angle (in degrees):  84.07763697853353
Angle (in radians):  1.4637688537455225
Angle (in degrees):  83.8677775023207
Angle (in radians):  1.5045035692362905
Angle (in degrees):  86.20170477960788
Angle (in radians):  1.4578338999301874
Angle (in degrees):  83.52772969709693
Angle (in radians):  1.4541731369361712
Angle (in degrees):  83.31798342774213
Angle (in radians):  1.4736455036453482
Angle (in degre

Angle (in radians):  1.4634519188686461
Angle (in degrees):  83.84961847149519
Angle (in radians):  1.5070530142622096
Angle (in degrees):  86.34777721969367
Angle (in radians):  1.474456833872816
Angle (in degrees):  84.48015365513432
Angle (in radians):  1.4991890119436302
Angle (in degrees):  85.89720307675798
Angle (in radians):  1.4760396282499804
Angle (in degrees):  84.57084109278287
Angle (in radians):  1.4904611205126437
Angle (in degrees):  85.39713173371405
Angle (in radians):  1.4994888620096063
Angle (in degrees):  85.91438322002513
Angle (in radians):  1.473234708690871
Angle (in degrees):  84.41013104017222
Angle (in radians):  1.474456833872816
Angle (in degrees):  84.48015365513432
Angle (in radians):  1.5086358479737663
Angle (in degrees):  86.4384669110369
Angle (in radians):  1.5083775167989393
Angle (in degrees):  86.42366562500266
Angle (in radians):  1.5086358479737663
Angle (in degrees):  86.4384669110369
Angle (in radians):  1.5086358479737663
Angle (in degrees

Angle (in radians):  -1.503157728831472
Angle (in degrees):  -86.12459380451361
Angle (in radians):  -1.4763072977787797
Angle (in degrees):  -84.58617742708734
Angle (in radians):  -1.470160244744212
Angle (in degrees):  -84.23397723176352
Angle (in radians):  -1.4266419396817422
Angle (in degrees):  -81.74056202012119
Angle (in radians):  -1.4320825265591113
Angle (in degrees):  -82.0522846862687
Angle (in radians):  -1.430735817416996
Angle (in degrees):  -81.97512393619381
Angle (in radians):  -1.414524700791628
Angle (in degrees):  -81.04629537236586
Angle (in radians):  -1.408255885510596
Angle (in degrees):  -80.68711871421561
Angle (in radians):  -1.408255885510596
Angle (in degrees):  -80.68711871421561
Angle (in radians):  -1.408255885510596
Angle (in degrees):  -80.68711871421561
Angle (in radians):  -1.4253342249056145
Angle (in degrees):  -81.66563548264217
Angle (in radians):  -1.459083882657053
Angle (in degrees):  -83.59934843181058
Angle (in radians):  -1.4739218303354

In [9]:
import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)  
time.sleep(1)
_, frame = cap.read()
x, y, w, h = cv2.selectROI(frame)
track_window = (x, y, w, h)

roi = frame[y:y + h, x:x + w]

hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

z_list = []  # List to store z-coordinates over time
x_list=[]
y_list=[]
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        print("points is :  ", pts)
        center = np.mean(pts, axis=0)
        z = center[1]  # z-coordinate is the y-coordinate of the center
        z_list.append(z)
        x_list.append(x)
        y_list.append(y)
        cv2.polylines(frame, [pts], True, (0, 255, 0), 2)
        cv2.circle(frame, (int(center[0]), int(center[1])), 5, (0, 0, 255), -1)
        angle = np.arctan2(pts[1][1]-pts[0][1], pts[1][0]-pts[0][0])
        angle_deg = np.degrees(angle)
        print("Angle (in radians): ", angle)
        print("Angle (in degrees): ", angle_deg)
       # Calculate velocity iny y-direction
        if len(y_list) >= 2:
            delta_t = 1/30  # Time between frames (assuming 30 fps)
            delta_y = y_list[-1] - y_list[-2]  # Change in z-coordinate
            vel_y = delta_y/delta_t
            print("Velocity in y-direction: ", vel_y)
        # Calculate velocity in x-direction
        if len(x_list) >= 2:
            delta_t = 1/30  # Time between frames (assuming 30 fps)
            delta_x = x_list[-1] - x_list[-2]  # Change in z-coordinate
            vel_x = delta_x/delta_t
            print("Velocity in x-direction: ", vel_x)
        
        # Calculate velocity in z-direction
        if len(z_list) >= 2:
            delta_t = 1/30  # Time between frames (assuming 30 fps)
            delta_z = z_list[-1] - z_list[-2]  # Change in z-coordinate
            vel_z = delta_z/delta_t
            print("Velocity in z-direction: ", vel_z)
        
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


points is :   [[203 326]
 [305 331]
 [302 409]
 [200 404]]
Angle (in radians):  0.04898040092372724
Angle (in degrees):  2.8063702517882496
points is :   [[157 313]
 [296 308]
 [299 415]
 [160 420]]
Angle (in radians):  -0.03595572032059952
Angle (in degrees):  -2.060111023723124
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -105.0
points is :   [[149 298]
 [301 292]
 [306 429]
 [154 435]]
Angle (in radians):  -0.03945320109703948
Angle (in degrees):  -2.260501911141272
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -15.0
points is :   [[298 265]
 [311 437]
 [163 448]
 [150 276]]
Angle (in radians):  1.495358360923359
Angle (in degrees):  85.67772294050896
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -210.0
points is :   [[293 254]
 [308 448]
 [163 460]
 [148 266]]
Angle (in radians):  1.4936302692073373
Angle (in degrees):  85.5787105785694
Velocity in y-direction:  

Angle (in radians):  1.1029907375984138
Angle (in degrees):  63.19671410641076
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  0.0
points is :   [[220 245]
 [316 436]
 [187 501]
 [ 91 310]]
Angle (in radians):  1.1050566706369513
Angle (in degrees):  63.3150833502756
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  45.0
points is :   [[224 243]
 [298 438]
 [165 489]
 [ 91 294]]
Angle (in radians):  1.2080975061338772
Angle (in degrees):  69.21888834175125
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -210.0
points is :   [[213 241]
 [318 428]
 [189 500]
 [ 84 313]]
Angle (in radians):  1.0591688678606777
Angle (in degrees):  60.68590592006641
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  135.0
points is :   [[216 246]
 [308 431]
 [177 496]
 [ 85 311]]
Angle (in radians):  1.109313216578321
Angle (in degrees):  63.55896546801962
Veloc

points is :   [[253 247]
 [328 437]
 [185 494]
 [110 304]]
Angle (in radians):  1.194835372923591
Angle (in degrees):  68.45902408146156
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -30.0
points is :   [[249 249]
 [324 433]
 [182 491]
 [107 307]]
Angle (in radians):  1.1837479869354262
Angle (in degrees):  67.82376365850723
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -15.0
points is :   [[258 250]
 [320 444]
 [175 491]
 [113 297]]
Angle (in radians):  1.2614674937202095
Angle (in degrees):  72.27676338311369
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  15.0
points is :   [[253 248]
 [328 435]
 [185 492]
 [110 305]]
Angle (in radians):  1.1893682909573258
Angle (in degrees):  68.14578335854249
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -15.0
points is :   [[272 256]
 [317 449]
 [165 484]
 [120 291]]
Angle (in radians):  1.

points is :   [[303 269]
 [308 463]
 [156 467]
 [151 273]]
Angle (in radians):  1.5450288353258959
Angle (in degrees):  88.5236314901869
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -60.0
points is :   [[293 268]
 [310 462]
 [162 475]
 [145 281]]
Angle (in radians):  1.4833907284314638
Angle (in degrees):  84.99202810795973
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  105.0
points is :   [[303 268]
 [308 462]
 [160 466]
 [155 272]]
Angle (in radians):  1.5450288353258959
Angle (in degrees):  88.5236314901869
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -135.0
points is :   [[301 266]
 [313 466]
 [162 475]
 [150 275]]
Angle (in radians):  1.5108681716736887
Angle (in degrees):  86.56636963754949
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  105.0
points is :   [[294 269]
 [315 462]
 [166 479]
 [145 286]]
Angle (in radians):  1

points is :   [[165 467]
 [166 257]
 [315 258]
 [314 468]]
Angle (in radians):  -1.5660344580257355
Angle (in degrees):  -89.72716501693192
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  15.0
points is :   [[163 467]
 [166 258]
 [316 259]
 [313 468]]
Angle (in radians):  -1.556443245521076
Angle (in degrees):  -89.17762902000182
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  15.0
points is :   [[310 258]
 [315 464]
 [170 467]
 [165 261]]
Angle (in radians):  1.5465292468134375
Angle (in degrees):  88.60959873595598
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -15.0
points is :   [[168 468]
 [172 256]
 [312 259]
 [308 471]]
Angle (in radians):  -1.5519306407732258
Angle (in degrees):  -88.91907581333932
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  30.0
points is :   [[165 469]
 [168 258]
 [310 260]
 [307 471]]
Angle (in radians):

...

points is :   [[166 449]
 [223 257]
 [358 296]
 [301 488]]
Angle (in radians):  -1.2822089649008193
Angle (in degrees):  -73.46516214265485
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -90.0
points is :   [[163 446]
 [226 255]
 [362 300]
 [299 491]]
Angle (in radians):  -1.252190416120696
Angle (in degrees):  -71.7452259904462
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  15.0
points is :   [[163 446]
 [226 255]
 [362 300]
 [299 491]]
Angle (in radians):  -1.252190416120696
Angle (in degrees):  -71.7452259904462
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  0.0
points is :   [[164 446]
 [223 255]
 [357 296]
 [298 487]]
Angle (in radians):  -1.2711940488377624
Angle (in degrees):  -72.83405394055085
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -60.0
points is :   [[169 458]
 [218 256]
 [352 289]
 [303 491]]
Angle (in radians): 

Angle (in radians):  -1.523661452981396
Angle (in degrees):  -87.29937066260472
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -45.0
points is :   [[160 465]
 [170 253]
 [319 260]
 [309 472]]
Angle (in radians):  -1.523661452981396
Angle (in degrees):  -87.29937066260472
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  0.0
points is :   [[164 470]
 [171 254]
 [315 259]
 [308 475]]
Angle (in radians):  -1.5384002574295246
Angle (in degrees):  -88.14384195255113
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  60.0
points is :   [[164 466]
 [175 254]
 [322 261]
 [311 473]]
Angle (in radians):  -1.518956023155807
Angle (in degrees):  -87.02976939280349
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -30.0
points is :   [[158 458]
 [179 250]
 [325 265]
 [304 473]]
Angle (in radians):  -1.470175746937771
Angle (in degrees):  -84.2348654420276

Angle (in radians):  -1.2049567150001825
Angle (in degrees):  -69.03893426545844
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  75.0
points is :   [[199 444]
 [273 241]
 [395 286]
 [321 489]]
Angle (in radians):  -1.22123449696225
Angle (in degrees):  -69.97158247171907
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  120.0
points is :   [[197 447]
 [267 242]
 [392 285]
 [322 490]]
Angle (in radians):  -1.2417466319545571
Angle (in degrees):  -71.1468412355809
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  30.0
points is :   [[189 441]
 [266 244]
 [391 293]
 [314 490]]
Angle (in radians):  -1.1981914533681672
Angle (in degrees):  -68.65131332664217
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  30.0
points is :   [[184 442]
 [266 242]
 [384 291]
 [302 491]]
Angle (in radians):  -1.1816990957396183
Angle (in degrees):  -67.706370840305

Angle (in radians):  -1.2733570910324703
Angle (in degrees):  -72.95798712921632
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  30.0
points is :   [[167 441]
 [226 242]
 [359 281]
 [300 480]]
Angle (in radians):  -1.2825697933473088
Angle (in degrees):  -73.48583608976696
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -90.0
points is :   [[165 443]
 [226 242]
 [361 283]
 [300 484]]
Angle (in radians):  -1.27614756869345
Angle (in degrees):  -73.117869722016
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  60.0
points is :   [[165 440]
 [227 243]
 [361 285]
 [299 482]]
Angle (in radians):  -1.2658895085468245
Angle (in degrees):  -72.53012616962299
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -15.0
points is :   [[166 440]
 [228 244]
 [360 286]
 [298 482]]
Angle (in radians):  -1.2644291741788496
Angle (in degrees):  -72.446455173660

points is :   [[167 442]
 [221 242]
 [359 279]
 [305 479]]
Angle (in radians):  -1.3070844923326306
Angle (in degrees):  -74.89042487765954
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -75.0
points is :   [[163 438]
 [227 243]
 [360 286]
 [296 481]]
Angle (in radians):  -1.2536682356229127
Angle (in degrees):  -71.82989881080535
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  45.0
points is :   [[164 444]
 [221 242]
 [352 279]
 [295 481]]
Angle (in radians):  -1.2957689074194103
Angle (in degrees):  -74.24208961941011
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -15.0
points is :   [[166 450]
 [219 244]
 [349 277]
 [296 483]]
Angle (in radians):  -1.318976265275508
Angle (in degrees):  -75.57177327821428
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  60.0
points is :   [[167 454]
 [222 243]
 [355 278]
 [300 489]]
Angle (in radian

In [15]:
import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)  
time.sleep(1)
_, frame = cap.read()
x, y, w, h = cv2.selectROI(frame)
track_window = (x, y, w, h)

roi = frame[y:y + h, x:x + w]

hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

z_list = []  # List to store z-coordinates over time
x_list=[]
y_list=[]
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        print("points is :  ", pts)
        center = np.mean(pts, axis=0)
        z = center[1]  # z-coordinate is the y-coordinate of the center
        z_list.append(z)
        x = int(center[0])
        y = int(center[1])
        x_list.append(x)
        y_list.append(y)
        cv2.drawContours(frame, [pts], 0, (0, 255, 0), 2)
        cv2.circle(frame, (int(center[0]), int(center[1])), 5, (0, 0, 255), -1)
        angle = np.arctan2(pts[1][1]-pts[0][1], pts[1][0]-pts[0][0])
        angle_deg = np.degrees(angle)
        print("Angle (in radians): ", angle)
        print("Angle (in degrees): ", angle_deg)
       # Calculate velocity iny y-direction
        if len(y_list) >= 2:
            delta_t = 1/30  # Time between frames (assuming 30 fps)
            delta_y = y_list[-1] - y_list[-2]  # Change in z-coordinate
            vel_y = delta_y/delta_t
            print("Velocity in y-direction: ", vel_y)
        # Calculate velocity in x-direction
        if len(x_list) >= 2:
            delta_t = 1/30  # Time between frames (assuming 30 fps)
            delta_x = x_list[-1] - x_list[-2]  # Change in z-coordinate
            vel_x = delta_x/delta_t
            print("Velocity in x-direction: ", vel_x)
        
        # Calculate velocity in z-direction
        if len(z_list) >= 2:
            delta_t = 1/30  # Time between frames (assuming 30 fps)
            delta_z = z_list[-1] - z_list[-2]  # Change in z-coordinate
            vel_z = delta_z/delta_t
            print("Velocity in z-direction: ", vel_z)
        
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
           


points is :   [[282 321]
 [283 258]
 [327 259]
 [326 322]]
Angle (in radians):  -1.5549246438031066
Angle (in degrees):  -89.09061955080087
points is :   [[250 348]
 [288 270]
 [351 301]
 [313 379]]
Angle (in radians):  -1.1174576309403337
Angle (in degrees):  -64.02560603756868
Velocity in y-direction:  1020.0
Velocity in x-direction:  -120.0
Velocity in z-direction:  1035.0
points is :   [[232 348]
 [275 258]
 [355 297]
 [312 387]]
Angle (in radians):  -1.125084014166261
Angle (in degrees):  -64.46256560936368
Velocity in y-direction:  -60.0
Velocity in x-direction:  -210.0
Velocity in z-direction:  -60.0
points is :   [[205 363]
 [256 250]
 [358 296]
 [307 409]]
Angle (in radians):  -1.146839054291001
Angle (in degrees):  -65.70903759164904
Velocity in y-direction:  210.0
Velocity in x-direction:  -360.0
Velocity in z-direction:  210.0
points is :   [[183 377]
 [232 250]
 [356 297]
 [307 424]]
Angle (in radians):  -1.2025676410472514
Angle (in degrees):  -68.90205041101085
Velocity 

Angle (in radians):  1.3110352135927745
Angle (in degrees):  75.1167845318984
Velocity in y-direction:  -150.0
Velocity in x-direction:  30.0
Velocity in z-direction:  -135.0
points is :   [[324 198]
 [363 422]
 [210 448]
 [171 224]]
Angle (in radians):  1.3984171173985271
Angle (in degrees):  80.12339882578617
Velocity in y-direction:  180.0
Velocity in x-direction:  30.0
Velocity in z-direction:  165.0
points is :   [[314 186]
 [367 415]
 [217 450]
 [164 221]]
Angle (in radians):  1.3433597237254349
Angle (in degrees):  76.9688425373277
Velocity in y-direction:  -150.0
Velocity in x-direction:  -60.0
Velocity in z-direction:  -150.0
points is :   [[324 192]
 [361 411]
 [206 438]
 [169 219]]
Angle (in radians):  1.40342707568001
Angle (in degrees):  80.41044829085175
Velocity in y-direction:  -90.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -90.0
points is :   [[319 187]
 [372 414]
 [219 449]
 [166 222]]
Angle (in radians):  1.3414251371478225
Angle (in degrees):  76.8579

Angle (in radians):  1.4301011946888826
Angle (in degrees):  81.93876273228983
Velocity in y-direction:  -60.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -60.0
points is :   [[333 190]
 [366 427]
 [213 448]
 [180 211]]
Angle (in radians):  1.4324453566177713
Angle (in degrees):  82.0730733173104
Velocity in y-direction:  0.0
Velocity in x-direction:  90.0
Velocity in z-direction:  0.0
points is :   [[339 192]
 [366 428]
 [213 446]
 [186 210]]
Angle (in radians):  1.4568848167836885
Angle (in degrees):  83.47335123839555
Velocity in y-direction:  0.0
Velocity in x-direction:  90.0
Velocity in z-direction:  0.0
points is :   [[332 190]
 [368 427]
 [218 450]
 [182 213]]
Angle (in radians):  1.420049945674475
Angle (in degrees):  81.36286858492926
Velocity in y-direction:  30.0
Velocity in x-direction:  -30.0
Velocity in z-direction:  30.0
points is :   [[334 193]
 [363 429]
 [214 448]
 [185 212]]
Angle (in radians):  1.448527922855778
Angle (in degrees):  82.99453648648777
Ve

Angle (in radians):  1.4448292971050174
Angle (in degrees):  82.7826208409708
Velocity in y-direction:  60.0
Velocity in x-direction:  -90.0
Velocity in z-direction:  60.0
points is :   [[341 197]
 [368 426]
 [217 444]
 [190 215]]
Angle (in radians):  1.4534342252534684
Angle (in degrees):  83.27564690689036
Velocity in y-direction:  -90.0
Velocity in x-direction:  60.0
Velocity in z-direction:  -75.0
points is :   [[337 193]
 [370 431]
 [219 452]
 [186 214]]
Angle (in radians):  1.433019319508044
Angle (in degrees):  82.10595896852016
Velocity in y-direction:  60.0
Velocity in x-direction:  -30.0
Velocity in z-direction:  60.0
points is :   [[342 199]
 [369 429]
 [215 447]
 [188 217]]
Angle (in radians):  1.4539398526565859
Angle (in degrees):  83.30461722309515
Velocity in y-direction:  30.0
Velocity in x-direction:  0.0
Velocity in z-direction:  15.0
points is :   [[345 199]
 [370 423]
 [212 440]
 [187 216]]
Angle (in radians):  1.459649149756238
Angle (in degrees):  83.631735850891

Angle (in radians):  1.4715563069392932
Angle (in degrees):  84.31396570347944
Velocity in y-direction:  -30.0
Velocity in x-direction:  90.0
Velocity in z-direction:  -30.0
points is :   [[343 202]
 [368 430]
 [210 447]
 [185 219]]
Angle (in radians):  1.4615834955650246
Angle (in degrees):  83.74256570185379
Velocity in y-direction:  120.0
Velocity in x-direction:  -60.0
Velocity in z-direction:  135.0
points is :   [[333 194]
 [381 419]
 [223 452]
 [175 227]]
Angle (in radians):  1.3606137401327272
Angle (in degrees):  77.95742485711503
Velocity in y-direction:  -30.0
Velocity in x-direction:  60.0
Velocity in z-direction:  -45.0
points is :   [[335 195]
 [375 425]
 [221 452]
 [181 222]]
Angle (in radians):  1.3986055122719576
Angle (in degrees):  80.13419305691563
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  15.0
points is :   [[338 196]
 [369 425]
 [218 446]
 [187 217]]
Angle (in radians):  1.4362430814153893
Angle (in degrees):  82.2906669

...

points is :   [[352 211]
 [383 428]
 [215 452]
 [184 235]]
Angle (in radians):  1.4288992721907328
Angle (in degrees):  81.86989764584403
Velocity in y-direction:  -30.0
Velocity in x-direction:  -90.0
Velocity in z-direction:  -30.0
points is :   [[354 209]
 [379 436]
 [214 453]
 [189 226]]
Angle (in radians):  1.461106223307875
Angle (in degrees):  83.71522001584043
Velocity in y-direction:  0.0
Velocity in x-direction:  30.0
Velocity in z-direction:  -15.0
points is :   [[357 212]
 [378 436]
 [212 451]
 [191 227]]
Angle (in radians):  1.477319545636307
Angle (in degrees):  84.64417495714481
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  15.0
points is :   [[357 208]
 [388 433]
 [221 456]
 [190 231]]
Angle (in radians):  1.4338805510397488
Angle (in degrees):  82.15530390047043
Velocity in y-direction:  30.0
Velocity in x-direction:  150.0
Velocity in z-direction:  15.0
points is :   [[341 202]
 [398 415]
 [235 458]
 [178 245]]
Angle (in radians

Angle (in radians):  1.373400766945016
Angle (in degrees):  78.69006752597979
Velocity in y-direction:  -150.0
Velocity in x-direction:  -30.0
Velocity in z-direction:  -135.0
points is :   [[347 204]
 [388 422]
 [222 453]
 [181 235]]
Angle (in radians):  1.3848945136600463
Angle (in degrees):  79.34861070354339
Velocity in y-direction:  90.0
Velocity in x-direction:  -60.0
Velocity in z-direction:  90.0
points is :   [[364 215]
 [375 436]
 [206 444]
 [195 223]]
Angle (in radians):  1.52106361375926
Angle (in degrees):  87.15052543932276
Velocity in y-direction:  30.0
Velocity in x-direction:  30.0
Velocity in z-direction:  30.0
points is :   [[349 204]
 [388 425]
 [222 454]
 [183 233]]
Angle (in radians):  1.396124127786657
Angle (in degrees):  79.99202019855866
Velocity in y-direction:  0.0
Velocity in x-direction:  0.0
Velocity in z-direction:  -15.0
points is :   [[351 207]
 [384 429]
 [220 454]
 [187 232]]
Angle (in radians):  1.423228255480902
Angle (in degrees):  81.544972322822

 [188 229]]
Angle (in radians):  1.4481276712904398
Angle (in degrees):  82.97160376105039
Velocity in y-direction:  -60.0
Velocity in x-direction:  -60.0
Velocity in z-direction:  -60.0
points is :   [[352 206]
 [378 436]
 [217 455]
 [191 225]]
Angle (in radians):  1.4582307109912116
Angle (in degrees):  83.55046529615772
Velocity in y-direction:  30.0
Velocity in x-direction:  60.0
Velocity in z-direction:  45.0
points is :   [[349 206]
 [378 429]
 [217 450]
 [188 227]]
Angle (in radians):  1.4414772252030152
Angle (in degrees):  82.59056126836167
Velocity in y-direction:  -60.0
Velocity in x-direction:  -30.0
Velocity in z-direction:  -75.0
points is :   [[347 205]
 [382 428]
 [221 453]
 [186 230]]
Angle (in radians):  1.4151156840064412
Angle (in degrees):  81.08015621633773
Velocity in y-direction:  30.0
Velocity in x-direction:  30.0
Velocity in z-direction:  30.0
points is :   [[347 204]
 [384 430]
 [222 457]
 [185 231]]
Angle (in radians):  1.4085191418469138
Angle (in degrees)

Angle (in radians):  1.5030269623956025
Angle (in degrees):  86.11710143963631
Velocity in y-direction:  60.0
Velocity in x-direction:  -90.0
Velocity in z-direction:  45.0
points is :   [[334 201]
 [387 423]
 [231 460]
 [178 238]]
Angle (in radians):  1.3364442562102954
Angle (in degrees):  76.57261543535039
Velocity in y-direction:  0.0
Velocity in x-direction:  90.0
Velocity in z-direction:  15.0
points is :   [[347 210]
 [375 437]
 [217 456]
 [189 229]]
Angle (in radians):  1.4480682288605033
Angle (in degrees):  82.96819796069103
Velocity in y-direction:  90.0
Velocity in x-direction:  0.0
Velocity in z-direction:  75.0
points is :   [[343 205]
 [382 429]
 [224 457]
 [185 233]]
Angle (in radians):  1.3984171173985271
Angle (in degrees):  80.12339882578617
Velocity in y-direction:  -60.0
Velocity in x-direction:  30.0
Velocity in z-direction:  -60.0
points is :   [[343 206]
 [388 425]
 [227 458]
 [182 239]]
Angle (in radians):  1.3681376576474205
Angle (in degrees):  78.38851357611

points is :   [[354 205]
 [394 428]
 [223 459]
 [183 236]]
Angle (in radians):  1.3933115553001871
Angle (in degrees):  79.83087166550933
Velocity in y-direction:  -120.0
Velocity in x-direction:  180.0
Velocity in z-direction:  -135.0
points is :   [[355 208]
 [384 426]
 [214 449]
 [185 231]]
Angle (in radians):  1.43854527525841
Angle (in degrees):  82.42257291079218
Velocity in y-direction:  -120.0
Velocity in x-direction:  -120.0
Velocity in z-direction:  -105.0
points is :   [[362 212]
 [389 431]
 [216 452]
 [189 233]]
Angle (in radians):  1.4481276712904398
Angle (in degrees):  82.97160376105039
Velocity in y-direction:  120.0
Velocity in x-direction:  150.0
Velocity in z-direction:  105.0
points is :   [[371 213]
 [386 442]
 [207 453]
 [192 224]]
Angle (in radians):  1.5053875827894092
Angle (in degrees):  86.25235502523395
Velocity in y-direction:  30.0
Velocity in x-direction:  0.0
Velocity in z-direction:  30.0
points is :   [[360 207]
 [397 426]
 [216 456]
 [179 237]]
Angle 